# Check GPU

In [1]:
#Check for GPU
import tensorflow as tf
# if tf.test.gpu_device_name():
#     print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
# else:
#     print('Please install GPU version of TF')
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [2]:
#check keras and tf versions
import keras
print(keras.__version__)
import tensorflow as tf
print(tf.__version__)

2.4.3
2.2.0


# Preprocessing

In [6]:
#create dicts from csv
##------------------------------ Dataset -------------------------------------##
import csv
import threading


#csv.field_size_limit(10**7)

def read_dict(path):
    'Reads Python dictionary stored in a csv file'
    dictionary = {}
    for key, val in csv.reader(open(path)):
        dictionary[key] = val
    return dictionary

# Load partitions
partition = read_dict('/autofs/space/euler_001/users/lstm/dicts/partition.csv')

def read_patterns(csv_file):
    '''
    Reads in target patterns of the form:
        label,v1,v2,...,vN
    '''
    pattern_dict = {}
    with open(csv_file,'r') as f:
        for line in f:
            atoms = line.strip().split(',')
            word = atoms[0]
            pattern = np.array([float(x) for x in atoms[1:]])
            pattern_dict[word] = pattern
    return pattern_dict

# Load dictionary of labels
labels = read_patterns('/autofs/space/euler_001/users/lstm/dicts/DorsalSparse.csv')
#print(labels)


exec("partition['train'] = " + partition['train'])
exec("partition['validation'] = " + partition['validation'])
exec("partition['test'] = " + partition['test'])

# # Final computations
# partition['train'] = partition['train'] 
# partition['validation'] = partition['validation']
# partition['test'] = partition['test']

partition['train'] = partition['train'] + partition['validation']
partition['validation'] = partition['test']

In [7]:
#Define the Data Generator
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    my_list = [] #global variable for getting the IDs
    my_labels = [] #global variable for getting the labels
    def __init__(self, list_IDs, labels, batch_size=100, dim=(max_len,211), #n_channels=1,
                 n_classes=300, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        #self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.my_list = []
        self.my_labels = []
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # FA and NS added this part to get the names of the words
        if index == 0:
            indexes2 = self.indexes[index*800:(index+1)*800]
            list_IDs_temp2 = [self.list_IDs[k] for k in indexes2]
            self.my_list = list_IDs_temp2 # This is for getting shuffled list_IDs for get_my_list
            
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))#, self.n_channels))
        #y = np.empty((self.batch_size), dtype=int)
        y = np.empty((self.batch_size, max_len, self.n_classes), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('Cochs/' + ID + '.npy') #path to cochleagrams stored in npy format

            # Store class
            y[i] = self.labels[ID]

        return X, y

# Build and Train LSTM

In [8]:
from keras import backend as K
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, LSTM, Masking, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import json
import pandas as pd
from keras.optimizers import Adam, SGD, RMSprop
from keras.metrics import categorical_crossentropy
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
# Parameters
params = {'dim': (max_len,211),
          'batch_size': 100,
          'n_classes': 300,
          #'n_channels': 1,
          'shuffle': True}

# Datasets
partition = partition
labels = labels

# Generators
training_generator = DataGenerator(partition['train'], labels, **params) #9 token per word
validation_generator = DataGenerator(partition['validation'], labels, **params) #1 token per word

In [10]:
data_dim = 211

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(Masking(mask_value=-9999, input_shape=(max_len, data_dim))) #The mask value has to equal whatever the junk value was that you used to pad the inputs. 0. is a bad idea because it might actually occur in the cgram. In EARSHOT we use -9999.
model.add(LSTM(512, return_sequences=True))
model.add(Dense(300, activation='sigmoid')) 
#bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
opt = keras.optimizers.Adam(learning_rate=0.001)#, clipnorm=0.01)
#sgd=SGD(lr=1e-3, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(loss='mse',
              optimizer=opt)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 226, 211)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 226, 512)          1482752   
_________________________________________________________________
dense (Dense)                (None, 226, 300)          153900    
Total params: 1,636,652
Trainable params: 1,636,652
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
# #load the model from checkpoint
# model = tf.keras.models.load_model('/autofs/space/euler_001/users/lstm/DorsalModel_CheckPoints/Mid1334/_1334.hd5f')
# print(model.summary())

In [12]:
# Early Stopping and checkpoint
# Set callback functions to early stop training and save the best model so far
# earlystop = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50),
#              ModelCheckpoint(filepath='VentralModel_EarlyStop.h5', monitor='val_loss', save_best_only=True)]

# checkpoint
checkpoints = ModelCheckpoint('checkpoints/' + '_{epoch:02d}' + '.hd5f',
                              monitor="loss",
                              verbose=0,
                              #save_best_only=True,
                              save_weights_only=False,
                              mode="min",
                              save_freq=7900)

from keras.callbacks import TerminateOnNaN
terminate = TerminateOnNaN()

In [13]:
# #Time-based decay
# from tensorflow.keras.callbacks import LearningRateScheduler
# initial_learning_rate = 0.0001
# epochs = 10000
# decay = initial_learning_rate / epochs

# def lr_time_based_decay(epoch, lr):
#     return lr * 1 / (1 + decay * epoch)

In [14]:
# #Step Decay
# from tensorflow.keras.callbacks import LearningRateScheduler
# # Define the learning rate schedule function
# initial_learning_rate = 0.01
# epochs = 10000

# def lr_step_decay(epoch, lr):
#     drop_rate = 0.0001
#     epochs_drop = 10.0
#     return initial_learning_rate * math.pow(drop_rate, math.floor(epoch/epochs_drop))

In [15]:
# #Exponential Decay
# from tensorflow.keras.callbacks import LearningRateScheduler
# initial_learning_rate = 0.01
# epochs = 10000
# def lr_exp_decay(epoch, lr):
#     k = 0.001
#     return initial_learning_rate * math.exp(-k*epoch)

In [ ]:
#Here we train the Network.
#history = model.fit(X_train, Y_train, batch_size=100, epochs=1000, validation_data=(X_valid, Y_valid))
history = model.fit_generator(generator=training_generator,
                              epochs=10000,
                              validation_data=validation_generator,
                              callbacks=[checkpoints],# LearningRateScheduler(lr_step_decay, verbose=1)], #earlystop,
                              #use_multiprocessing=True,
                              workers=6)

In [ ]:
#Plot loss
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.ylim([0, 1])
plt.legend(loc='upper left')